# Europe

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
#from pygeocoder import geocoder
import pycountry
import folium
import json

In [3]:
df=pd.DataFrame.from_csv('tsdec450.tsv', sep='\t', header=0)

In [4]:
#Create multi index for easy processing.
df["age"] = np.nan
df["unit"]=np.nan
df["sex"]=np.nan
df["geo/time"]=np.nan
for i,indexStr in enumerate(df.index):
    stringList=indexStr.split(',')
    df.iloc[i, df.columns.get_loc('age')] = stringList[0]
    df.iloc[i,df.columns.get_loc('unit')]=stringList[1]
    df.iloc[i,df.columns.get_loc('sex')]=stringList[2]
    df.iloc[i,df.columns.get_loc('geo/time')]=stringList[3]
                
df=df.set_index(['age', 'unit', 'sex','geo/time'])
#Now select the needed latest data 


In [24]:
#We want the data of the total age and the percentage of the active population of 2016
countryDf=df.loc[('TOTAL', 'PC_ACT', 'T')]["2016 "].copy()

#Dropping columns that do not correspond to a country
countryDf.drop(['EA18','EA19','EU27','EU28'],inplace=True)

#Greece is abbreviated as EL 
nameIndexes=[]

for countryCode in countryDf.index:
    if countryCode =="EL":
        countryCode="GR"
    if countryCode =="UK":
        countryCode="GB"    
    #nameIndexes.append(pycountry.countries.get(alpha_2=countryCode).name)
    nameIndexes.append(countryCode)

countrySeries=pd.Series(countryDf.values,nameIndexes)
test=pd.Series.to_frame(countrySeries)
test.reset_index(inplace=True)
test.columns = ['id', 'Unemployment']
test

,id,Unemployment
0,AT,6.0
1,BE,7.8
2,BG,7.6
3,CY,13.0
4,CZ,4.0
5,DE,4.1
6,DK,6.2
7,EE,6.8
8,GR,23.6
9,ES,19.6


In [25]:
#Plotting Distribution of the percentages

# h = test['Unemployment'].tolist()


# std = np.std(h) 
# mean = np.mean(h)    
# plt.plot(norm.pdf(h,mean,std))
# plt.show()


In [26]:
#Checking the Keys of the geojson:
from pprint import pprint

json_data=open('europe.topojson.json')
jdata = json.load(json_data)

for key, value in jdata.items():
    pprint("Key:")
    pprint(key)
    
    
    
print(jdata)

'Key:'
'type'
'Key:'
'objects'
'Key:'
'arcs'
'Key:'
'transform'
{'type': 'Topology', 'objects': {'europe': {'type': 'GeometryCollection', 'geometries': [{'type': 'MultiPolygon', 'properties': {'NAME': 'Azerbaijan'}, 'id': 'AZ', 'arcs': [[[0, 1, 2]], [[3]], [[4]], [[5, 6, 7, 8, 9, 10], [11]]]}, {'type': 'Polygon', 'properties': {'NAME': 'Albania'}, 'id': 'AL', 'arcs': [[12, 13, 14, 15, 16, 17, 18]]}, {'type': 'MultiPolygon', 'properties': {'NAME': 'Armenia'}, 'id': 'AM', 'arcs': [[[-12]], [[19, -3, 20, 21, -7], [-5], [-4]]]}, {'type': 'Polygon', 'properties': {'NAME': 'Bosnia and Herzegovina'}, 'id': 'BA', 'arcs': [[22, 23, 24, 25, 26, 27, 28, 29, 30, 31]]}, {'type': 'Polygon', 'properties': {'NAME': 'Bulgaria'}, 'id': 'BG', 'arcs': [[32, 33, 34, 35, 36, 37]]}, {'type': 'Polygon', 'properties': {'NAME': 'Cyprus'}, 'id': 'CY', 'arcs': [[38]]}, {'type': 'MultiPolygon', 'properties': {'NAME': 'Denmark'}, 'id': 'DK', 'arcs': [[[39]], [[40]], [[41]], [[42]], [[43]], [[44]], [[45]], [[46]], [

In [27]:
#europe = 'map-of-europe-master\TopoJSON\europe.topojson'
#europe = os.path.join('data', 'europe.geojson')

#folium.TopoJson(open(europe),'objects.europe',name='topojson').add_to(m)

In [28]:
#Choropleth
m = folium.Map(location=[46.519962, 6.633597], zoom_start=3)
topo_json_data = json.load(open('europe.topojson.json'))

m.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=test,
    columns=['id', 'Unemployment'],
    key_on='feature.id',
    #threshold_scale = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
    fill_color= 'PuBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    topojson = 'objects.europe'
)
#m.choropleth(geo_data=europe)
#folium.LayerControl().add_to(m)
m

# Switzerland

2)Just per canton

The data of the swiss cantons is collected from amstat.ch. We used the "taux de chômage" sorted by canton to get our data.

In [35]:
dfCanton=pd.read_excel('rateSuisseCantonsGerman.xlsx')
dfCanton.head()

,Kanton,Monat,Oktober 2016,November 2016,Dezember 2016,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,Gesamt
0,NaN,Metriken,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote
1,Zürich,NaN,3.6,3.7,3.8,3.9,3.9,3.8,3.6,3.5,3.4,3.4,3.4,3.3,3.6
2,Bern,NaN,2.6,2.8,3,3,3,2.9,2.7,2.6,2.4,2.4,2.5,2.4,2.7
3,Luzern,NaN,1.9,2,2.2,2.2,2.2,2,1.9,1.8,1.7,1.7,1.8,1.7,1.9
4,Uri,NaN,1,1.1,1.3,1.5,1.4,1.3,1.2,1,0.8,0.7,0.6,0.6,1.1


In [37]:
#The data frame looks a bit wierd. We clean it up by removing the first row and second column.
#We can see that the total column is the average af the past year.
#We will use this number to do further analysis since this average eliminates seasonal trends.
dfCanton=dfCanton[['Kanton','Gesamt']].iloc[1:]
#Since each canton has an unique name we can index by canton
dfCanton.set_index('Kanton',drop=True,inplace=True,verify_integrity=True)
dfCanton['Gesamt']=dfCanton['Gesamt'].astype(float) #Convert to float
dfCanton.reset_index(inplace=True)
dfCanton.columns = ['name', 'Unemployment']
dfCanton

,name,Unemployment
0,Bern,2.7
1,Luzern,1.9
2,Uri,1.1
3,Schwyz,1.8
4,Obwalden,0.9
5,Nidwalden,1.1
6,Glarus,2.2
7,Zug,2.4
8,Freiburg,2.7
9,Solothurn,2.9


In [38]:
# #Converting the Canton names from French to German to fit the json
# import gettext
# #german = gettext.translation('Appenzell Rhodes-Extérieures', pycountry.LOCALES_DIR, languages=['de'])
# #german = pycountry.countries.lookup('Genève')
# #german
# canti= list(pycountry.subdivisions.get(country_code='CH'))
# canti2 = pycountry.subdivisions.get(code='CH-OW')
# type(canti2)

# canton_code_translate = pd.DataFrame()

# for i in canti:
#     i_str = str(i)
#     #print (i_str)
#     code = i_str.replace("Subdivision(code='","")
#     code = code[:5]
#     #print(code)
#     name = i_str[51:]
#     name = name.replace("', parent_code=None, type='Canton')","")
#     #print(name)
   


In [47]:
#Choropleth2
s = folium.Map(location=[46.519962, 6.633597], zoom_start=10)
topo_json_data = json.load(open('ch-cantons.topojson.json'))

In [48]:
s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=dfCanton,
    columns=['name', 'Unemployment'],
    key_on='feature.properties.name',
    #threshold_scale = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
    fill_color= 'BuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    topojson ='objects.cantons'
)
#folium.LayerControl().add_to(m)
s

3) Include the information about suisse and foreign workers

,Kanton,Nationalität,Monat,Oktober 2016,November 2016,Dezember 2016,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,Gesamt
0,NaN,NaN,Metriken,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote
1,Zürich,Ausländer,NaN,5.8,6.1,6.4,6.6,6.5,6.3,6,5.7,5.6,5.5,5.3,5.3,5.9
2,Zürich,Schweizer,NaN,2.8,2.8,2.9,2.9,2.8,2.8,2.7,2.6,2.6,2.6,2.6,2.5,2.7
3,Bern,Ausländer,NaN,6.3,6.9,7.3,7.5,7.4,7.1,6.8,6.2,5.7,5.5,5.5,5.5,6.5
4,Bern,Schweizer,NaN,2,2,2.1,2.2,2.1,2.1,2,1.9,1.8,1.8,1.9,1.8,2


,Canton,Nationality,Unemployement
1,Zürich,Ausländer,5.9
2,Zürich,Schweizer,2.7
3,Bern,Ausländer,6.5
4,Bern,Schweizer,2.0
5,Luzern,Ausländer,4.5


In [126]:
dfCantonForeign=pd.read_excel('rateSuisseCantonForeignersGerman.xlsx')
dfCantonForeign.head()
#Same process as before but with hierachal index
dfCantonForeign=dfCantonForeign[['Kanton','Nationalität','Gesamt']].iloc[1:-1]#Also leave out last column
dfCantonForeign.columns = ['Canton', 'Nationality','Unemployement']
dfCantonForeign["Unemployement"]=dfCantonForeign["Unemployement"].astype(float)
dfCantonForeign.head()
#We have to show the difference between the two percentages
seriesCantonForeingSuisse=pd.Series()
for canton,cantonDf in dfCantonForeign.groupby("Canton"):
    unEmployementForeign=cantonDf.loc[cantonDf['Nationality'] == "Ausländer"].Unemployement.values[0]
    unEmployementSwiss=cantonDf.loc[cantonDf['Nationality'] == "Schweizer"].Unemployement.values[0]
    difference=unEmployementForeign-unEmployementSwiss
    seriesCantonForeingSuisse[canton]=difference
dfCantonForeign=seriesCantonForeingSuisse.to_frame()
dfCantonForeign.reset_index(inplace=True)
dfCantonForeign.columns=["Canton","unemployementDifference"]


,Canton,unemployementDifference
0,Aargau,4.0
1,Appenzell Ausserrhoden,2.3
2,Appenzell Innerrhoden,3.0
3,Basel-Landschaft,3.0
4,Basel-Stadt,2.4
5,Bern,4.5
6,Freiburg,3.6
7,Genf,1.3
8,Glarus,3.0
9,Graubünden,3.3


In [128]:
#vizualize it the same way as before
s = folium.Map(location=[46.519962, 6.633597], zoom_start=10)
topo_json_data = json.load(open('ch-cantons.topojson.json'))
s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=dfCantonForeign,
    columns=['Canton', 'unemployementDifference'],
    key_on='feature.properties.name',
    #threshold_scale = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],
    fill_color= 'BuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment difference between foreigners-suisse (%)',
    topojson ='objects.cantons'
)
#folium.LayerControl().add_to(m)
s

4) Now also with age categories

In [79]:
dftem=pd.read_excel('rateSuisseCantonAgeGerman.xlsx')
dftem.head(4)

,Kanton,Altersklassen in 5-Jahresschritten,Monat,Oktober 2016,November 2016,Dezember 2016,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,Gesamt
0,NaN,NaN,Metriken,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote
1,Zürich,15-19 Jahre,NaN,4,3.8,3.6,3.5,3.3,3,2.7,2.5,2.4,4,4.2,3.9,3.4
2,Zürich,20-24 Jahre,NaN,4.3,4.2,4.3,4.5,4.3,4.1,3.7,3.5,3.4,3.4,3.6,3.7,3.9
3,Zürich,25-29 Jahre,NaN,4.2,4.3,4.5,4.6,4.4,4.2,4,3.9,3.7,3.6,3.6,3.5,4


In [83]:
#Same
dfCantonAge=pd.read_excel('rateSuisseCantonAgeGerman.xlsx')
dfCantonAge.columns.values[1] = 'Agegroup'
dfCantonAge=dfCantonAge[['Kanton','Agegroup','Gesamt']].iloc[1:]
for index,agegroup in enumerate(dfCantonAge["Agegroup"]):
    #Remove the ans
    cleaned=str(agegroup).split()[0]
    if cleaned=="60":
        cleaned=cleaned+"+"
    dfCantonAge.iloc[index, dfCantonAge.columns.get_loc('Agegroup')]=cleaned
    

dfCantonAge.set_index(['Kanton','Agegroup'],drop=True,inplace=True,verify_integrity=True)
#In the datarame the unkown values are rplaced with ... 
#We replace the ... with NaN
dfCantonAge.replace("...",np.nan,inplace=True)
dfCantonAge["Gesamt"]=dfCantonAge["Gesamt"].astype(float)


dfCantonAge.head(15)


Gesamt
Kanton Agegroup        
Zürich 15-19        3.4
       20-24        3.9
       25-29        4.0
       30-34        4.1
       35-39        4.1
       40-44        3.4
       45-49        3.2
       50-54        3.4
       55-59        3.3
       60+          2.6
Bern   15-19        2.0
       20-24        3.3
       25-29        3.5
       30-34        3.3
       35-39        3.1